<a href="https://colab.research.google.com/github/kikiymini/7506R-1C2024-GRUPO02/blob/main/7506R_1C2024_GRUPO02_N2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Organizaci&oacute;n de Datos</center>

#### <center>C&aacute;tedra Ing. Rodriguez, Juan Manuel </center>

## <center>Trabajo Práctico 1: Propiedades en venta</center>

### <center> Grupo 2</center>

## Integrantes:

*   Aramayo Carolina
*   Utrera Maximo Damian
*   Villalba Ana Daniela
*   Fiorilo Roy


# Importación de librerias

In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk

from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import folium
from folium.plugins import MarkerCluster

from shapely import wkt
from shapely.geometry import MultiPolygon, Polygon, Point

from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale, MinMaxScaler
from sklearn.metrics import silhouette_score
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder

# Lectura de archivo

### Desde google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/7506R-1C2024-GRUPO02/Dataset/properati_argentina_2021.csv'
barrios_file = '/content/drive/MyDrive/7506R-1C2024-GRUPO02/Dataset/barrios.csv'
df = pd.read_csv(file_path)

### Desde maquina local

In [ ]:
# Si se esta trabajando en local, descomentar esta celta y comentar la de arriba
# file_path = './dataset/properati_argentina_2021.csv'
# barrios_file = './dataset/barrios.csv'
# df = pd.read_csv(file_path)

# Agrupamiento
En este punto se busca analizar si es posible agrupar los datos en función de algún criterio.

## Clustering
Analizar la tendencia al clustering del dataset.

In [ ]:
ds_train.head()

Para agrupar, podemos dejar de lado la columna 'place_l3', ya que podemos completar los datos necesarios para realizar el clustering utilizando las columnas de latitud y longitud. Además, podemos eliminar la columna 'id', ya que generalmente solo sirve como un identificador único para cada fila y no es una característica relevante para el clustering. Similarmente, podemos descartar cualquier información relacionada con fechas, ya que no es necesaria para nuestro análisis de clustering.

In [ ]:
ds_cluster_train = ds_train.copy()
ds_cluster_test = ds_test.copy()

columnas_a_eliminar = ['place_l3','id','start_date','end_date','created_on']
ds_cluster_train.drop(columns = columnas_a_eliminar,inplace=True)
ds_cluster_test.drop(columns = columnas_a_eliminar,inplace=True)

In [ ]:
ds_cluster_train.head()

In [ ]:
# @title property_type vs longitud
# Configuración de estilo y paleta de colores
sns.set(style="whitegrid")
palette = sns.color_palette("pastel")

# Ajuste del tamaño de la figura
figsize = (12, 1.2 * len(ds_cluster_train['property_type'].unique()))
plt.figure(figsize=figsize)

# Creación del gráfico de violín
sns.violinplot(ds_cluster_train, x='longitud', y='property_type', inner='box', palette=palette)

# Eliminación de los ejes
sns.despine(top=True, right=True, bottom=True, left=True)

# Ajustes de título y etiquetas
plt.title("Distribución de Longitud por Tipo de Propiedad", fontsize=16)
plt.xlabel("Longitud", fontsize=14)
plt.ylabel("Tipo de Propiedad", fontsize=14)

# Ajuste del tamaño de fuente de las etiquetas
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.show()

In [ ]:
# @title latitud vs longitud

# Configuración de la figura y los ejes
plt.figure(figsize=(10, 6))

# Creación del gráfico de dispersión
plt.scatter(ds_cluster_train['latitud'], ds_cluster_train['longitud'], s=50, alpha=0.8)

# Ajustes del título y etiquetas
plt.title("Distribución de Latitud vs Longitud", fontsize=16)
plt.xlabel("Latitud", fontsize=14)
plt.ylabel("Longitud", fontsize=14)

# Ocultar los ejes superior y derecho
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Ajustar tamaño de fuente de los ejes
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.show()

In [ ]:
ds_cluster_test.head()

Vamos a realizar una transformación de variables para trabajar con todos datos numéricos. En este caso, vamos a utilizar la regresión logística para la variable proterty_type

In [ ]:
ds_cluster_train = pd.get_dummies(ds_cluster_train, columns=["property_type"])
ds_cluster_train.head()

In [ ]:
ds_cluster_test = pd.get_dummies(ds_cluster_test, columns=["property_type"])
ds_cluster_test.head()

Aplico máximos y minimos para el dataset

In [ ]:
def aplicar_max_min(ds, columnas):
    scaler = MinMaxScaler()
    for columna in columnas:
        ds[[columna]] = scaler.fit_transform(ds[[columna]])

aplicar_max_min(ds_cluster_train,columnas_con_numeros)

## Grupos
Estimar la cantidad apropiada de grupos que se deben formar.

Para ello vamos a utilizar la Regla del codo (Elbow Method).

In [ ]:
# Corro el algoritmo con K entre 2 y 10 y voy guardando el error para luego graficarlo.
sse = []
list_k = list(range(1, 10))

for k in list_k:
    km = KMeans(n_clusters=k)
    km.fit(ds_cluster_train)
    sse.append(km.inertia_)

# Grafico el SSE por K
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse, '-o')
plt.xlabel(r'Cantidad de clusters')
plt.ylabel('SSE')

# Desactivar la notación científica en el eje y
plt.ticklabel_format(style='plain', axis='y')

plt.show()

Al ver el gráfico no tenemos un codo pronunciadamente. Entonces verificamos la cantidad de conjuntos a elegir con el método de Silhouette.

## Grupos con Silhouette <font color='red'> REVISAR </font>
Evaluar la calidad de los grupos formados realizando un análisis de Silhouette

In [ ]:
list_k = list(range(2, 6))

for n_clusters in list_k:
    clusterer = KMeans(n_clusters=n_clusters)
    preds = clusterer.fit_predict(ds_cluster_train)

    score = silhouette_score (ds_cluster_train, preds)
    print ("\n\nFor n_clusters =" + str(n_clusters) + " silhouette score is " + str(score))

## Análisis de grupo
Realizar un análisis de cada grupo intentando entender en función de qué características
fueron formados.

## Mapa de CABA
Graficar sobre un mapa de CABA los avisos coloreados según el grupo al que pertenecen.

In [ ]:
# Coordenadas del centro de la Ciudad Autónoma de Buenos Aires
latitud = -34.6037
longitud = -58.3816

# Crear el mapa centrado en Buenos Aires
mapa = folium.Map(location=[latitud, longitud], zoom_start=12)

ds_latitud_longitud = ds_cluster_train[['latitud','longitud']]
ds_latitud_longitud.shape

In [ ]:
# OJO QUE ACÁ ESTOY DISMINUYENDO LA CANTIDAD DE PROPIEDADES

ds_latitud_longitud = ds_latitud_longitud.sample(n=1000)

# Iterar sobre las filas del DataFrame ds_latitud_longitud
for index, row in ds_latitud_longitud.iterrows():
    # Obtener las coordenadas de cada punto
    punto_latitud = row['latitud']
    punto_longitud = row['longitud']

    # Añadir un marcador para cada punto
    folium.Marker(location=[punto_latitud, punto_longitud]).add_to(mapa)

# Mostrar el mapa
mapa

In [ ]:
mapa_cluster = folium.Map(location=[latitud, longitud], zoom_start=12)

marcador = MarkerCluster()

for index, row in ds_latitud_longitud.iterrows():
    # Obtener las coordenadas de cada punto
    punto_latitud = row['latitud']
    punto_longitud = row['longitud']

    # Añadir un marcador para cada punto
    marcador.add_child(folium.Marker(location=[punto_latitud, punto_longitud]))

mapa_cluster.add_child(marcador)


## Análisis con tres grupos
Repetir el análisis anterior, utilizando sólo 3 grupos.